In [ ]:
# Kiwi 형태소분석기 설치
!pip install kiwipiepy

# colab에서 한글폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845033 sha256=2cd5b7e8e453d2feee55af09294944d985cdc1de60e19acb24146eec78dc83ca
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (43.6 MB/s)
debconf: unable to initialize f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm.notebook import tqdm
from collections import Counter
from itertools import combinations

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

import re # 정규표현식 모듈
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
kiwi = Kiwi()

import networkx as nx # 네트워크 생성을 위한 라이브러리

In [ ]:
#냄새 관련 표현 정리파일(xlsx) 불러오기
data = pd.read_excel('/content/drive/MyDrive/201_[Lecture] Tutoring/허수빈2/data/소설 taxt/냄새 정리.xlsx', header = 1)

In [ ]:
data['sentence'][0]

'할머니(노인) 미워. 영택이 삼촌 방은 더럽고 냄새도 나빠. 그러니까 삼촌이 도망가버렸잖아.'

#### 네트워크

In [ ]:
words_list = []
coocurrences = []
for sentence in data['sentence']:
    words = []
    for token in kiwi.tokenize(sentence):
        word, tag = token.form, token.tag
        if tag in ["NNG", 'VA-I', 'VA', 'VV', 'VX']:
            if tag.startswith('V'):
                word += '다'
            words.append(word)
            words_list.append(word)

    comb_words = combinations(list(set(words)), 2)


    # words_list.append(words)
    coocurrences.extend(comb_words)

In [ ]:
threshold_4_words = [k for k, v in dict(Counter(words_list)).items() if v >= 4]

In [ ]:
coocurrences = [tuple(sorted(co)) for co in coocurrences]

In [ ]:
len(threshold_4_words)

59

In [ ]:
coocurrences = []
for idx, row in data.iterrows():

    coocurrences.append((row['position'], row['subject']))
    coocurrences.append((row['subject'], row['character']))

coocurrences = [tuple(sorted(co)) for co in coocurrences]

In [ ]:
coocurrences

In [ ]:
G = nx.Graph() # 네트워크 초기화

for word1, word2 in tqdm(coocurrences):
    G.add_edge(word1, word2)

nx.write_gexf(G, "냄새_network4.gexf") #지금까지 작업을 게피형태(gexf)로 저장

  0%|          | 0/174 [00:00<?, ?it/s]

In [ ]:
[(n1, n2) for n1, n2 in coocurrences if n1 == '노인' or n2 == '노인']

[('노인', '로션'),
 ('긍정', '노인'),
 ('노인', '위액'),
 ('노인', '부정'),
 ('노인', '노인'),
 ('노인', '부정'),
 ('노인', '부정'),
 ('노인', '노인'),
 ('노인', '부정'),
 ('노인', '부정'),
 ('노인', '할아버지'),
 ('노인', '부정'),
 ('노인', '할아버지'),
 ('노인', '부정'),
 ('노인', '배변'),
 ('노인', '부정'),
 ('노인', '배변'),
 ('노인', '부정')]